# Torsion and Warping Properties of a Built-up Section
In this notebook we compare the torsion and warping properties of individual sections of a built-up section, with the properties of the combined section.

## Import Modules
Start by importing the necessary modules.

In [ ]:
from sectionproperties.pre.library.steel_sections import i_section, angle_section
from sectionproperties.analysis.section import Section
%config InlineBackend.figure_formats = ['svg']

## Create Geometry
Here we create a 200UB25 I-section and place two 100x75x6 unequal angles either side of the centre of the web.

In [ ]:
# create 200UB25 I-section, align centroid at origin (0, 0)
i_geom = i_section(d=203, b=133, t_f=7.8, t_w=5.8, r=8.9, n_r=16).align_center()

# create 100x75x6UA
r_angle_geom = (
    angle_section(d=100, b=75, t=6, r_r=8, r_t=5, n_r=16)
    .mirror_section(axis="x")  # flange on top
    .shift_section(x_offset=2.9, y_offset=-50)  # centred on RHS of web
)

# mirror RHS angle to LHS of web
l_angle_geom = r_angle_geom.mirror_section(axis="y", mirror_point=(0, 0))

## I-Section Section Properties 
Calculate the torsion and warping constant for the I-section alone.

In [ ]:
i_geom.create_mesh(3.0)
i_sec = Section(i_geom)
i_sec.plot_mesh()
i_sec.calculate_geometric_properties()
i_sec.calculate_warping_properties()
i_j = i_sec.get_j()
i_iw = i_sec.get_gamma()

In [ ]:
print(f"J={i_j:.3e}; Iw={i_iw:.3e}")

## Angle Section Properties
Calculate the torsion and warping constant for one of the angles alone.

In [ ]:
r_angle_geom.create_mesh(3.0)
angle_sec = Section(r_angle_geom)
angle_sec.plot_mesh()
angle_sec.calculate_geometric_properties()
angle_sec.calculate_warping_properties()
angle_j = angle_sec.get_j()
angle_iw = angle_sec.get_gamma()

In [ ]:
print(f"J={angle_j:.3e}; Iw={angle_iw:.3e}")

## Built-up Section Properties
Calculate the torsion and warping constant of the built-up section.

In [ ]:
bup_geom = i_geom + r_angle_geom + l_angle_geom
bup_geom.create_mesh(3.0)
bup_sec = Section(bup_geom)
bup_sec.plot_mesh()
bup_sec.calculate_geometric_properties()
bup_sec.calculate_warping_properties()
bup_j = bup_sec.get_j()
bup_iw = bup_sec.get_gamma()

In [ ]:
print(f"J={bup_j:.3e}; Iw={bup_iw:.3e}")

## Interpreting the Results
Let's compare the sum of the individual properties to the built-up properties.

In [ ]:
(i_j + 2 * angle_j) / bup_j

In [ ]:
(i_iw + 2 * angle_iw) / bup_iw

## Stress Due to Torsion
We can also visualise the stress due to a unit twisting moment.

In [ ]:
stress_i = i_sec.calculate_stress(Mzz=1e6)
stress_angle = angle_sec.calculate_stress(Mzz=1e6)
stress_bup = bup_sec.calculate_stress(Mzz=1e6)

In [ ]:
stress_i.plot_stress_mzz_zxy()
stress_angle.plot_stress_mzz_zxy()
stress_bup.plot_stress_mzz_zxy()